# Unpacking of JSON data from DiesDas Indexer Tool

In [ ]:
import pandas as pd
import json
from ast import literal_eval

In [ ]:
root = 'C:/Projekte/ENC/neues Tool/'
src = root + '00_raw_data/'
dst = root + '01_unpacked_data/'
file = 'raw_IPN_Dachau_Kartei_7_8_1.csv'

### load functions

In [ ]:
def get_all_attributes(json_objs, attributes):
    for record in json_objs:
        if type(record) == str:
            record = json.loads(record.replace('\\\\"','"'))
            
        keys = list (record.keys())
        for key in keys:
            if '_repeat' in key:
                get_all_attributes(record[key],attributes)
                
            else:
                attributes.add ((key))
            

In [ ]:
def extract_data_from_one_obj(json_obj, output):
    
    keys = list (json_obj.keys())
    for key in keys:
        if "_repeat" in key:
            for obj in json_obj[key]:
                extract_data_from_one_obj(obj, output)
        else:
            if key in result_set:
                result_set[key].append (json_obj[key])
            else:
                result_set[key] = []
                result_set[key].append (json_obj[key])

In [ ]:
def xstr(s):
    return '' if s is None else str(s)


def join_stringed_list(column: pd.Series):
    return column.apply(
        lambda x:', '.join(
            [
                xstr(v) for v in x
            ]
        )
    )


def unlist_columns(df):
    
    for col_name in df:
        try:
            df[col_name] = join_stringed_list(df[col_name])
        except:
            pass
    return df

### load data

In [ ]:
origin_df = pd.read_csv(src+file)

In [ ]:
origin_df.head()

In [ ]:
json_data = origin_df['json_data']

In [ ]:
atts = set()
get_all_attributes(json_data, atts)

In [ ]:
atts

In [ ]:
# create a dictionary of lists, attributes are a keys of dict
attributes_as_dic = {el:[] for el in atts}

### unpack json data

In [ ]:

for i, record in enumerate(json_data):
    print(i, end='\r')
    result_set = {}
    #record = json.loads(record)
    record = json.loads(record.replace('\\\\"','"'))
    extract_data_from_one_obj(record, result_set)
    for attribute in attributes_as_dic:
        if attribute in result_set:
            attributes_as_dic[attribute].append(result_set[attribute])
        else:
            attributes_as_dic[attribute].append("-")

In [ ]:
output_df = pd.DataFrame(attributes_as_dic)

### seperate multiple dates per date type

In [ ]:
#enter all date columns with multiple date entries
columns = ['birthdate_month', 'birthdate_year', 'birthdate_day']

for i, row in output_df.iterrows():
    
    for column in columns:
    
        for n, element in enumerate(row[column][:2]):
            # print(i, column, element)

            output_df.loc[i, f'{column}_{n}'] = element

In [ ]:
output_df

### Handle multiple persons per classification

If a field called "person_list" is present, unse the following to deal with it

In [ ]:
for i, row in output_df.iterrows():
    if i%25==0:
        print(i, end='\r')
    
    for n, number_dict in enumerate(row['person_list'][0]):
        
        keys = number_dict.keys()
        
        for key in keys:
            
            output_df.loc[i, f'{key}_{n}'] = number_dict[key]
        if n == 11:
            break

In [ ]:
def join_list_columns(a,b):
    new_list = []
    for i, value in enumerate(b):
        new_list.append(xstr(a[i])+xstr(b[i]))
        
    return new_list

In [ ]:
# Specifically for Central Name Index (CNI) workflows

output_df['person_list'] = pd.Series(list(map(join_list_columns,
                output_df['letter'],
                output_df['number'])),
          index=output_df.index)

In [ ]:
output_df = unlist_columns(output_df)

In [ ]:
output_df.head()

In [ ]:
output_df = output_df.drop(columns,axis=1)

In [ ]:
output_df.head()

In [ ]:
sorted_df = output_df.reindex(sorted(output_df.columns), axis=1)

In [ ]:
sorted_df

In [ ]:
export_df = origin_df.join(sorted_df)

## Export data

In [ ]:
export_df.to_csv(dst+'unpacked_'+'_'.join(file.split('_')[1:]), encoding='utf-8', index=False)